# 1.d LLM Agents

In this notebook you will see:
- How to create an agent with several tools
- How to merge everything we saw previously in an agent
- How to have a conversation with an agent

Note: For this notebook, we enable logging to see the calls.

# Setup

In [2]:
from typing import Any
import json


from conversational_toolkit.llms.base import LLMMessage, Roles
from conversational_toolkit.tools.base import Tool
from conversational_toolkit.agents.base import QueryWithContext
from conversational_toolkit.agents.tool_agent import ToolAgent
from conversational_toolkit.llms.openai import OpenAILLM

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


# Create an agent with several tools

We can wrap the process shown for the tool calling in a wrapper to simplify the use, it will also the agent to reflect.

## Define the Tools

In [4]:
class SumTwoNumbers(Tool):
    def __init__(
        self,
        name: str,
        description: str,
        parameters: dict[str, Any],
    ):
        self.name = name
        self.description = description
        self.parameters = parameters

    async def call(self, args: dict[str, Any]) -> dict[str, Any]:
        number_1 = args.get("number_1")
        number_2 = args.get("number_2")

        if not isinstance(number_1, (int, float)) or not isinstance(
            number_2, (int, float)
        ):
            raise ValueError("Both number_1 and number_2 must be int or float.")

        result = number_1 + number_2

        return {"result": result}

In [5]:
tool_sum = SumTwoNumbers(
    name="sum_two_numbers",
    description="A tool to sum two numbers. It takes two numbers as input and returns their sum.",
    parameters={
        "type": "object",
        "properties": {
            "number_1": {
                "type": "number",
                "description": "The first number to be summed.",
            },
            "number_2": {
                "type": "number",
                "description": "The second number to be summed.",
            },
        },
        "required": ["number_1", "number_2"],
        "additionalProperties": False,
    },
)

print(await tool_sum.call({"number_1": 5, "number_2": 10}))

{'result': 15}


In [6]:
class HalfBoldText(Tool):
    def __init__(
        self,
        name: str,
        description: str,
        parameters: dict[str, Any],
    ):
        self.name = name
        self.description = description
        self.parameters = parameters

    async def call(self, args: dict[str, Any]) -> dict[str, Any]:
        text = args.get("text")

        if not isinstance(text, str):
            raise ValueError("The 'text' parameter must be a string.")

        words = text.split()
        if len(words) < 2:
            raise ValueError("The input text must contain at least two words.")

        # Bold every other word: yes, no, yes, no...
        bolded_text = " ".join(
            [f"**{word}**" if i % 2 == 0 else word for i, word in enumerate(words)]
        )

        return {"result": bolded_text}


tool_half_bold = HalfBoldText(
    name="half_bold_text",
    description="A tool to bold every other word in a given text. It takes a string as input and returns the modified string with every other word bolded.",
    parameters={
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The input text to be modified.",
            },
        },
        "required": ["text"],
        "additionalProperties": False,
    },
)

print(
    await tool_half_bold.call(
        {"text": "This is an example of the half bold text tool in action."}
    )
)

{'result': '**This** is **an** example **of** the **half** bold **text** tool **in** action.'}


## Define the Agent

We will make use of the `ToolAgent` class we developed.

Bonus: Below you can find it's implementation, but it's not required to understand deeply.

```python
class ToolAgent(Agent):
    async def answer_stream(self, query_with_context: QueryWithContext) -> AsyncGenerator[AgentAnswer, None]:
        steps = []
        sources: list[Chunk] = []
        messages = [
            LLMMessage(role=Roles.SYSTEM, content=self.system_prompt),
            *query_with_context.history,
            LLMMessage(role=Roles.USER, content=query_with_context.query),
        ]

        while True:
            tool_calls: list[ToolCall] = []
            content = ""
            response_stream = self.llm.generate_stream(messages)
            async for response_chunk in response_stream:
                if response_chunk.content:
                    content += response_chunk.content
                    answer = await self._answer_post_processing(
                        AgentAnswer(content=content, role=Roles.ASSISTANT, sources=sources.copy())
                    )
                    if answer:
                        yield answer
                if response_chunk.tool_calls:
                    tool_calls += response_chunk.tool_calls

            steps.append(
                {
                    "content": content,
                    "tool_calls": tool_calls,
                    "role": Roles.ASSISTANT,
                    "function_name": "llm",
                }
            )
            messages.append(
                LLMMessage(
                    role=Roles.ASSISTANT,
                    content=content,
                    tool_calls=tool_calls,
                )
            )

            if not tool_calls:
                break

            available_functions = {tool.name: tool.call for tool in self.llm.tools} if self.llm.tools else {}

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = {
                    "_query": query_with_context.query,
                    "_history": query_with_context.history,
                    **json.loads(tool_call.function.arguments),
                }
                function_response = await function_to_call(function_args)
                if "_sources" in function_response:
                    sources += [
                        Chunk(
                            title=chunk.get("title") or "",
                            content=chunk.get("content") or "",
                            mime_type=chunk.get("mime_type") or "",
                            metadata=chunk.get("metadata") or {},
                        )
                        for chunk in function_response.get("_sources", [])
                    ]
                messages.append(self.build_tool_answer(tool_call.id, function_name, function_response))
                steps.append({**function_response, "role": "tool", "function_name": function_name})

            if len(steps) > self.max_steps:
                # TODO: Maybe throw an exception here?
                yield AgentAnswer(content="Request is too complex to execute", role=Roles.ASSISTANT)
                break

        logger.debug(steps)
```

In [7]:
# Create a custom ToolAgent
class MyAgent(ToolAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


# Define the LLM that will be it's core, and provide it with tools
llm = OpenAILLM(tool_choice="auto", tools=[tool_sum, tool_half_bold])

# Define the prompt
prompt = "You are a helpful assistant, answer shortly."

2026-02-26 15:11:19.326 | DEBUG    | conversational_toolkit.llms.openai:__init__:63 - OpenAI LLM loaded: gpt-4o-mini; temperature: 0.5; seed: 42; tools: [<__main__.SumTwoNumbers object at 0x000002090345F050>, <__main__.HalfBoldText object at 0x0000020903374B30>]; tool_choice: auto; response_format: {'type': 'text'}


In [8]:
# Instantiate the agent
simple_agent = MyAgent(
    system_prompt=prompt,
    llm=llm,
    # Maximum number of steps the agent can take (avoid infinite loops)
    max_steps=5,
)

## Test the Agent

Now it can select which tool if any to use, and iterate in calls.

Our implementation of the agent no longer take `LLMMessage` as input, but `QueryWithContext`, which is basically the user query and the list of previous message (it simplifies code further on).

```python
class QueryWithContext(BaseModel):
    query: str
    history: list[LLMMessage]
```

### No Tool Needed

In [9]:
# Similar to messages, but with history in case it's relevant
query = QueryWithContext(query="How are you?", history=[])

answer = await simple_agent.answer(query)

print(answer.content)

2026-02-26 15:11:20.465 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': "I'm just a program, but I'm here and ready to help you! How can I assist you today?", 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


I'm just a program, but I'm here and ready to help you! How can I assist you today?


### Calculator needed

Look into the logs, the calculator is called.

In [10]:
query = QueryWithContext(query="Sum 2+59", history=[])

answer = await simple_agent.answer(query)

print(answer.content)

2026-02-26 15:11:22.275 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '', 'tool_calls': [ToolCall(id='call_Kael7pVjHSBOo52EB1d8ezKE', function=Function(name='sum_two_numbers', arguments='{"number_1":2,"number_2":59}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': 61, 'role': 'tool', 'function_name': 'sum_two_numbers'}, {'content': 'The sum of 2 and 59 is 61.', 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


The sum of 2 and 59 is 61.


### Text Tool needed

Look into the logs, the text formatter is called.

In [11]:
query = QueryWithContext(
    query="Rewrite the following text with every other word bolded: 'This is an example of the half bold text tool in action.'",
    history=[],
)

answer = await simple_agent.answer(query)

print("Content: ", answer.content)

2026-02-26 15:11:24.429 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '', 'tool_calls': [ToolCall(id='call_GsI4xxBIAVRAJoBGRTytk8Ou', function=Function(name='half_bold_text', arguments='{"text":"This is an example of the half bold text tool in action."}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': '**This** is **an** example **of** the **half** bold **text** tool **in** action.', 'role': 'tool', 'function_name': 'half_bold_text'}, {'content': '**This** is **an** example **of** the **half** bold **text** tool **in** action.', 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


Content:  **This** is **an** example **of** the **half** bold **text** tool **in** action.


### Both Tools

Look into the logs, first the calculator is called, and then the text formatter.

In [12]:
query = QueryWithContext(
    query="First sum 10 and 20, then rewrite the result with every other word bolded by writing a short story about the result (1 sentence).",
    history=[],
)

answer = await simple_agent.answer(query)

print("Content: ", answer.content)

2026-02-26 15:11:29.446 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '', 'tool_calls': [ToolCall(id='call_PcpVgIyiiFYxRCkNsAIgHnjz', function=Function(name='sum_two_numbers', arguments='{"number_1":10,"number_2":20}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': 30, 'role': 'tool', 'function_name': 'sum_two_numbers'}, {'content': '', 'tool_calls': [ToolCall(id='call_DuynGgVf4LqozjJFs2csKyoF', function=Function(name='half_bold_text', arguments='{"text":"Once upon a time, in a land where numbers danced, thirty was the magical age when dreams began to take flight."}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': '**Once** upon **a** time, **in** a **land** where **numbers** danced, **thirty** was **the** magical **age** when **dreams** began **to** take **flight.**', 'role': 'tool', 'function_name': 'half_bold_text'}, {'content': '**Once** upon **

Content:  **Once** upon **a** time, **in** a **land** where **numbers** danced, **thirty** was **the** magical **age** when **dreams** began **to** take **flight.**


# Complete Agent

Let's put everything we have see together in one agent.

## Define it

In [13]:
model = "gpt-4.1-mini"
temperature = 0.7

complex_prompt = """You are a helpful assistant, you are responsible for answering questions with clear, professional, and well-supported responses. You must answer like you were a pirate. Justify your tools usage. Keep it short overall."""

In [14]:
output_schema = {
    "type": "object",
    "description": "The structured output for the answer.",
    "properties": {
        "answer": {
            "type": "string",
            "description": "The answer to the user's question in markdown format.",
        },
        "explanations": {
            "type": "array",
            "description": "List of justifications for the answer (max 3).",
            "items": {"type": "string"},
            "minItems": 1,
            "maxItems": 3,
        },
    },
    "required": ["answer", "explanations"],
    "additionalProperties": False,
}

response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "AnswerSchema",
        "schema": output_schema,
        "strict": True,
    },
}

In [15]:
tools = [tool_sum, tool_half_bold]
tool_choice = "auto"

In [16]:
llm = OpenAILLM(
    model_name=model,
    temperature=temperature,
    response_format=response_format,
    tool_choice=tool_choice,
    tools=tools,
)

2026-02-26 15:11:29.705 | DEBUG    | conversational_toolkit.llms.openai:__init__:63 - OpenAI LLM loaded: gpt-4.1-mini; temperature: 0.7; seed: 42; tools: [<__main__.SumTwoNumbers object at 0x000002090345F050>, <__main__.HalfBoldText object at 0x0000020903374B30>]; tool_choice: auto; response_format: {'type': 'json_schema', 'json_schema': {'name': 'AnswerSchema', 'schema': {'type': 'object', 'description': 'The structured output for the answer.', 'properties': {'answer': {'type': 'string', 'description': "The answer to the user's question in markdown format."}, 'explanations': {'type': 'array', 'description': 'List of justifications for the answer (max 3).', 'items': {'type': 'string'}, 'minItems': 1, 'maxItems': 3}}, 'required': ['answer', 'explanations'], 'additionalProperties': False}, 'strict': True}}


In [17]:
max_nb_steps = 5

In [18]:
complex_agent = MyAgent(
    system_prompt=complex_prompt,
    llm=llm,
    max_steps=max_nb_steps,
)

## Test it

### No Tool Needed

In [19]:
query = QueryWithContext(query="How are you?", history=[])

answer = await complex_agent.answer(query)
answer_as_json = json.loads(answer.content)

print("Answer: ", answer_as_json["answer"])
print("Explanations: ", answer_as_json["explanations"])

2026-02-26 15:11:31.973 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '{"answer":"Arrr! I\'m sailin\' the digital seas fine, matey! How can I be of service to ye today?","explanations":["Responded in pirate style as requested.","Kept the tone friendly and engaging.","Offered further assistance to the user."]}', 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


Answer:  Arrr! I'm sailin' the digital seas fine, matey! How can I be of service to ye today?
Explanations:  ['Responded in pirate style as requested.', 'Kept the tone friendly and engaging.', 'Offered further assistance to the user.']


### Calculator needed

In [20]:
query = QueryWithContext(query="Sum 2+59", history=[])

answer = await complex_agent.answer(query)
answer_as_json = json.loads(answer.content)

print("Answer: ", answer_as_json["answer"])
print("Explanations: ", answer_as_json["explanations"])

2026-02-26 15:11:34.432 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '', 'tool_calls': [ToolCall(id='call_UKlD893aILSrTkeLifXl4NnC', function=Function(name='sum_two_numbers', arguments='{"number_1":2,"number_2":59}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': 61, 'role': 'tool', 'function_name': 'sum_two_numbers'}, {'content': '{"answer":"Arrr, the sum of 2 and 59 be 61, matey!","explanations":["Used me trusty addition tool to sum the two numbers 2 and 59.","The result be a simple arithmetic sum, no tricks from the depths of the sea."]}', 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


Answer:  Arrr, the sum of 2 and 59 be 61, matey!
Explanations:  ['Used me trusty addition tool to sum the two numbers 2 and 59.', 'The result be a simple arithmetic sum, no tricks from the depths of the sea.']


### Text Tool needed

In [21]:
query = QueryWithContext(
    query="Rewrite the following text with every other word bolded: 'This is an example of the half bold text tool in action.'",
    history=[],
)

answer = await complex_agent.answer(query)
answer_as_json = json.loads(answer.content)

print("Answer: ", answer_as_json["answer"])
print("Explanations: ", answer_as_json["explanations"])

2026-02-26 15:11:37.135 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '', 'tool_calls': [ToolCall(id='call_5c6apTxvZFIXpMwpVTSpjMun', function=Function(name='half_bold_text', arguments='{"text":"This is an example of the half bold text tool in action."}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': '**This** is **an** example **of** the **half** bold **text** tool **in** action.', 'role': 'tool', 'function_name': 'half_bold_text'}, {'content': '{"answer":"**This** is **an** example **of** the **half** bold **text** tool **in** action.","explanations":["Used the half_bold_text function to accurately bold every other word as requested.","This function ensures the formatting is consistent and visually clear.","The tool automates the task, saving time and avoiding manual errors."]}', 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


Answer:  **This** is **an** example **of** the **half** bold **text** tool **in** action.
Explanations:  ['Used the half_bold_text function to accurately bold every other word as requested.', 'This function ensures the formatting is consistent and visually clear.', 'The tool automates the task, saving time and avoiding manual errors.']


### Both Tools

In [22]:
# create example where first the sum tool is used, then the half bold tool is used on the result
query = QueryWithContext(
    query="First sum 10 and 20, then rewrite the result with every other word bolded by writing a short story about the result (1 sentence).",
    history=[],
)

answer = await complex_agent.answer(query)
answer_as_json = json.loads(answer.content)

print("Answer: ", answer_as_json["answer"])
print("Explanations: ", answer_as_json["explanations"])

2026-02-26 15:11:42.312 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '', 'tool_calls': [ToolCall(id='call_xBb5pXyhho0Sic5EFeA4dWlG', function=Function(name='sum_two_numbers', arguments='{"number_1": 10, "number_2": 20}'), type='function'), ToolCall(id='call_LwYE4n1G5Mou630xQGoQyHOG', function=Function(name='half_bold_text', arguments='{"text": "The treasure chest held the mighty sum of thirty gold coins, gleaming brightly under the moonlit sky."}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': 30, 'role': 'tool', 'function_name': 'sum_two_numbers'}, {'result': '**The** treasure **chest** held **the** mighty **sum** of **thirty** gold **coins,** gleaming **brightly** under **the** moonlit **sky.**', 'role': 'tool', 'function_name': 'half_bold_text'}, {'content': '{"answer":"**The** treasure **chest** held **the** mighty **sum** of **thirty** gold **coins,** gleaming **brightly** under **the** moo

Answer:  **The** treasure **chest** held **the** mighty **sum** of **thirty** gold **coins,** gleaming **brightly** under **the** moonlit **sky.**
Explanations:  ['First, I summed 10 and 20 to get 30.', 'Then, I wrote a short story sentence using the number 30 as the amount of gold coins.', 'Finally, I bolded every other word in the story as requested.']


# Conversation with an Agent

In this implementation, only the final answer is kept in the history (this comes from our implementation of `ToolAgent`), but other implementation of `BaseAgent` could handle this differently.

In [23]:
# Let's create th history message from the query
user_initial_message = LLMMessage(role=Roles.USER, content=query.query)
history_messages = [user_initial_message, answer]

# And create a new query
new_query = QueryWithContext(
    query="Make a joke based on past conversation.", history=history_messages
)

In [24]:
final_answer = await complex_agent.answer(new_query)
final_answer_as_json = json.loads(final_answer.content)

print("Answer: ", final_answer_as_json["answer"])
print("Explanations: ", final_answer_as_json["explanations"])

2026-02-26 15:11:43.829 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '{"answer":"Why did the pirate bring a calculator to the treasure hunt? Because he wanted to make sure his 30 gold coins added up to a *bold* fortune!","explanations":["The joke references the earlier conversation about summing numbers and bolding text.","It ties in the pirate theme with treasure and gold coins.","It plays on the idea of counting and emphasizing the amount of treasure."]}', 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


Answer:  Why did the pirate bring a calculator to the treasure hunt? Because he wanted to make sure his 30 gold coins added up to a *bold* fortune!
Explanations:  ['The joke references the earlier conversation about summing numbers and bolding text.', 'It ties in the pirate theme with treasure and gold coins.', 'It plays on the idea of counting and emphasizing the amount of treasure.']


------------------------